In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import skimage
from skimage import io, feature, measure, transform
import tensorflow as tf
from pandas import Series,DataFrame
import sampling as smp

In [2]:
sess=tf.InteractiveSession()

In [3]:
### x will contain flattened 35 x 35 pixel patches as rows
### placeholder None is there to allow us to have arbitrary number of training ex.
### y will contain 1-hot patch labels


x= tf.placeholder(tf.float32,shape=[None,1225])
x_image = tf.reshape(x, [-1,35,35,1])
y_ = tf.placeholder(tf.float32, shape=[None, 50])

In [4]:
### helper fn for random positive initialization of weights
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [ ]:
### helper functions to build Conv and MaxPool layers

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x,poolSize,strideSize):
  return tf.nn.max_pool(x, ksize=[1, poolSize, poolSize, 1],
                        strides=[1, strideSize, strideSize, 1], padding='SAME')

In [158]:
### weights and biases for the first Conv layer; 5x5 filter and 32 feat.
W_conv1 = weight_variable([4, 4, 1, 32])
b_conv1 = bias_variable([32])

In [161]:
### build first Conv and MaxPool layers
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool(h_conv1,3,2)

In [162]:
h_pool1.get_shape()

TensorShape([Dimension(None), Dimension(18), Dimension(18), Dimension(32)])

In [163]:
### weights and biases for the second Conv layer; 5x5 filter and  feat.
W_conv2 = weight_variable([4, 4, 32, 64])
b_conv2 = bias_variable([64])

In [164]:
### build second Conv and MaxPool layers
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool(h_conv2,3,2)

In [165]:
h_pool2.get_shape()

TensorShape([Dimension(None), Dimension(9), Dimension(9), Dimension(64)])

In [166]:
### weights and biases for the third Conv layer; 5x5 filter and 128 feat.
W_conv3 = weight_variable([4, 4, 64, 128])
b_conv3 = bias_variable([128])

In [167]:
### build third Conv and MaxPool layers
h_conv3 = tf.nn.relu(conv2d(h_pool2,W_conv3)+b_conv3)
h_pool3 =max_pool(h_conv3,3,2)

In [168]:
h_pool3.get_shape()

TensorShape([Dimension(None), Dimension(5), Dimension(5), Dimension(128)])

In [169]:
### add fully-connected layer

In [170]:
W_fc = weight_variable([5 * 5 * 128, 128])
b_fc = bias_variable([128])

h_pool3_flat = tf.reshape(h_pool3, [-1, 5*5*128])
h_fc = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc) + b_fc)

In [171]:
### dropout before output layer

keep_prob = tf.placeholder(tf.float32)
h_fc_drop = tf.nn.dropout(h_fc, keep_prob)

In [172]:
### output layer

W_out = weight_variable([128,50])
b_out = bias_variable([50])

y_conv = tf.nn.softmax(tf.matmul(h_fc_drop, W_out) + b_out)

In [173]:
y_conv.get_shape()

TensorShape([Dimension(None), Dimension(50)])

In [174]:
####  cross entropy cost function
softmax_cost_fn = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.clip_by_value(y_conv,1e-10,1.0)), reduction_indices=[1]))

In [175]:
### positive-shared cost function from the DeepContour paper

modified_cost_fn = softmax_cost_fn-tf.reduce_mean(y_[:,0]*tf.log(tf.clip_by_value(y_conv,1e-10,1.0))[:,0] + tf.reduce_sum(tf.transpose(tf.transpose(y_[:,1:])*tf.log(tf.clip_by_value(y_conv,1e-10,1.0)[:,0])),reduction_indices=[1]))

In [176]:
#### the training step
train_step = tf.train.AdamOptimizer(1e-4).minimize(modified_cost_fn)

In [177]:
#### accuracy
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [178]:
#### contrast score from eqn (7) in DeepContour paper. This is broken but can't figure out how...

contrast = tf.reduce_mean((y_[:,0] - tf.reduce_sum(y_[:,1:],reduction_indices=[1])) * (2*y_conv[:,0]-1))

In [179]:
#### build a batch for the optimizer

def build_batch(X,y,b_size):
    m = X.shape[0]
    pts = np.random.randint(0,m,size=b_size)
    batch = (X[pts].flatten().reshape(b_size,35*35), y[pts].flatten().reshape(b_size,50))
    return batch

In [180]:
########              Training the network               ########
####  the input image patches are in 'ultra_training.msg'     ###
####  the cluster labels are in 'permuted_cluster_label_training.msg'  ###

In [181]:
#### load the training data
X = pd.read_msgpack('ultra_training.msg').as_matrix()[:6000,:] 
labels = pd.read_msgpack('permuted_cluster_label_training.msg').as_matrix()[:6000,:]

In [182]:
#### start the session
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver()

In [183]:
for i in range(10000):
    batch = build_batch(X,labels,50)
    if i%100 == 0:
        train_acc = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1],  keep_prob: 1.0})
        train_contrast = contrast.eval(feed_dict={
        x:batch[0], y_: batch[1],  keep_prob: 1.0})
        print("step %d, training contrast %g"%(i, train_contrast))
    train_step.run(feed_dict={
        x:batch[0], y_: batch[1],  keep_prob: 0.6})


step 0, training contrast -0.12
step 100, training contrast 0.12
step 200, training contrast -0.32
step 300, training contrast -0.04
step 400, training contrast -0.16
step 500, training contrast -0.32
step 600, training contrast -0.28
step 700, training contrast -0.12
step 800, training contrast -0.12
step 900, training contrast -0.2
step 1000, training contrast -0.24
step 1100, training contrast -0.16
step 1200, training contrast -0.04
step 1300, training contrast -0.24
step 1400, training contrast -0.16
step 1500, training contrast -0.16
step 1600, training contrast -0.12
step 1700, training contrast -0.32
step 1800, training contrast -0.16
step 1900, training contrast -0.08
step 2000, training contrast -0.24
step 2100, training contrast -0.24
step 2200, training contrast -0.12
step 2300, training contrast -0.16
step 2400, training contrast -0.2
step 2500, training contrast -0.28
step 2600, training contrast -0.28
step 2700, training contrast -0.16
step 2800, training contrast -0.24


KeyboardInterrupt: 

In [153]:
#save_path = saver.save(sess, "/Users/gus/CDIPS/uns/patch_trained_1.ckpt")

In [49]:
test_output = y_conv.eval(feed_dict={x:X[6000:7000,:],y_:labels[6000:7000,:],keep_prob:1.0})

In [55]:
test_accuracy = accuracy.eval(feed_dict={x:X[6000:7000,:],y_:labels[6000:7000,:],keep_prob:1.0})